# **MLCB - Assignment 2**
### Glykeria Spyrou

### **TASK 3: Nested Cross Validation implementation**

#### *3.2 Hyper-parameter tuning through n-CV*

In [1]:
import os
os.chdir('/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/')
import numpy as np
import pandas as pd
import optuna
from scipy import stats
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

from nestedCV import NestedCV
from functions import dict_to_df, metric_box_plots, evaluate_clfs, plot_median_scores
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from optuna.distributions import CategoricalDistribution, FloatDistribution
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import logging
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="OptunaSearchCV is experimental")
optuna.logging.disable_default_handler()

In [2]:
# Load data
os.chdir('/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/')
data = pd.read_csv('data/Diabetes.csv')
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

Generating the seeds to be used for the inner and outer loops of the nested cross validation.

In [3]:
np.random.seed(42)
rounds = 10
seeds = [np.random.randint(1,1000) for _ in range(3*rounds)]
print(seeds)

[103, 436, 861, 271, 107, 72, 701, 21, 615, 122, 467, 215, 331, 459, 88, 373, 100, 872, 664, 131, 662, 309, 770, 344, 492, 414, 806, 386, 192, 956]


In [4]:
# Define classifiers
classifiers = {
    'LogisticRegression': LogisticRegression(),
    'GaussianNB': GaussianNB(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(),
    'SVM': SVC(probability=True)
}

# Define parameter spaces for each classifier
param_grids = {
    'LogisticRegression': {
        'C': FloatDistribution(1e-4, 1e4),
        'penalty': CategoricalDistribution(['l1', 'l2']),
        'solver': CategoricalDistribution(['liblinear', 'saga'])
    },
    'GaussianNB': {'var_smoothing':FloatDistribution(1e-9, 1e-2)},
    'KNeighborsClassifier': {
        'n_neighbors': CategoricalDistribution(list(range(1, 31))),
        'weights': CategoricalDistribution(['uniform', 'distance']),
        'algorithm': CategoricalDistribution(['auto', 'ball_tree', 'kd_tree', 'brute']),
    },
    'LinearDiscriminantAnalysis': {
        'solver': CategoricalDistribution(['eigen', 'svd', 'lsqr'])
    },
    'SVM': {
        'C': FloatDistribution(1e-2, 1e2), 
        'gamma': FloatDistribution(1e-3, 1e-1) 
    }
}

##### *Perform nCV on the raw dataset*

In [5]:
# Initialize the NestedCV class with the classifiers, parameter grids, rounds, seeds, and folds
nested_cv = NestedCV(classifiers=classifiers, param_grids=param_grids,
                     rounds=rounds, inner_cv_folds=3,
                     outer_cv_folds=5, seeds=seeds,
                     n_trials=50)

# Fit the nested cross-validation
outer_loop_scores, outer_params = nested_cv.fit(X, y)

# Print median performance for each classifier
print("Median performance per classifier:")
for clf_name in classifiers.keys():
    print(f"\nClassifier: {clf_name}")
    median_performance = {metric: np.median([result[metric] for result in outer_loop_scores[clf_name]])
                          for metric in outer_loop_scores[clf_name][0]}
    
    # Print median performance and best parameters
    print(f"Median performance: {median_performance}")



Using classifier: LogisticRegression



  0%|          | 0/10 [00:00<?, ?it/s]


Starting round 1...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/Users/


Starting round 2...


...


Starting round 9...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/Users/glykeriasp/Documents/DSIT/Machine Le


Starting round 10...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(estimator=clf, param_distributions=param_grid,
/Users/

Median performance per classifier:

Classifier: LogisticRegression
Median performance: {'MCC': 0.49840717726632755, 'Balanced Accuracy': 0.7345959595959597, 'F1': 0.6506631299734749, 'F2': 0.6024096385542167, 'Recall': 0.5714285714285714, 'Precision': 0.7257053291536051, 'Average Precision': 0.5712011854246648, 'Specificity': 0.8787878787878788, 'NPV': 0.7974358974358975, 'PR-AUC': 0.7136905802892575}

Classifier: GaussianNB
Median performance: {'MCC': 0.4574115491070494, 'Balanced Accuracy': 0.7175324675324675, 'F1': 0.6259328358208955, 'F2': 0.591715976331361, 'Recall': 0.5714285714285714, 'Precision': 0.68, 'Average Precision': 0.5431398316218083, 'Specificity': 0.8615384615384616, 'NPV': 0.7891030392883618, 'PR-AUC': 0.6963687641702208}

Classifier: KNeighborsClassifier
Median performance: {'MCC': 0.4092578739947413, 'Balanced Accuracy': 0.6765526140526141, 'F1': 0.5555555555555556, 'F2': 0.493845977975079, 'Recall': 0.4646825396825397, 'Precision': 0.7171428571428571, 'Average Pre

In [6]:
# Construct DFs per model out of the NestedCV output for the score metrics of each outer loop across all rounds executed
logreg_scores = dict_to_df(outer_loop_scores, 'LogisticRegression')
gnb_scores = dict_to_df(outer_loop_scores, 'GaussianNB')
knn_scores = dict_to_df(outer_loop_scores, 'KNeighborsClassifier')
lda_scores = dict_to_df(outer_loop_scores, 'LinearDiscriminantAnalysis')
svm_scores = dict_to_df(outer_loop_scores, 'SVM')

# Construct DFs per model out of the NestedCV output for the saved hyper-parameters across all rounds executed
logreg_params = dict_to_df(outer_params, 'LogisticRegression')
gnb_params = dict_to_df(outer_params, 'GaussianNB')
knn_params = dict_to_df(outer_params, 'KNeighborsClassifier')
lda_params = dict_to_df(outer_params, 'LinearDiscriminantAnalysis')
svm_params = dict_to_df(outer_params, 'SVM')

Creating plots for the visualization of the metric distributions acroos all ten rounds of nested cross validation for the given classifiers.

In [7]:
# Store all classifier data-frames in a list
all_dfs = [logreg_scores, gnb_scores, knn_scores, lda_scores, svm_scores]

# Store their corresponding names
classifier_names = ['Logistic Regression', 'Gaussian NB', 'k-NN', 'LDA', 'SVM']


In [8]:
scores_fig = metric_box_plots(all_dfs, classifier_names, 'Box Plots of Metric Scores Per Classifier', [-0.1, 1])
pio.show(scores_fig)

In [9]:
# Save as pdf
pio.write_image(scores_fig, "plots/scores-fig.pdf", height=1200, width=450)

#### *Perform nCV on the raw dataset with balanced classes*

##### Undersampling Method

In [10]:
# Create a RandomUnderSampler object
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# Balancing the data (using the curated dataset)
X_u, y_u = rus.fit_resample(X, y)

Perform hyper-parameter tuning through n-CV

In [11]:
# Initialize the NestedCV class with the classifiers, parameter grids, rounds, seeds, and folds
nested_cv = NestedCV(classifiers=classifiers, param_grids=param_grids,
                     rounds=rounds, inner_cv_folds=3,
                     outer_cv_folds=5, seeds=seeds,
                     n_trials=50)

# Fit the nested cross-validation
outer_loop_scores_u, outer_params_u = nested_cv.fit(X_u, y_u)

# Print median performance for each classifier
print("Median performance per classifier:")
for clf_name in classifiers.keys():
    print(f"\nClassifier: {clf_name}")
    median_performance = {metric: np.median([result[metric] for result in outer_loop_scores_u[clf_name]])
                          for metric in outer_loop_scores_u[clf_name][0]}
    
    # Print median performance and best parameters
    print(f"Median performance: {median_performance}")


Using classifier: LogisticRegression



  0%|          | 0/10 [00:00<?, ?it/s]


Starting round 1...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 2...


...


Starting round 9...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 10...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t

Median performance per classifier:

Classifier: LogisticRegression
Median performance: {'MCC': 0.5512273465625287, 'Balanced Accuracy': 0.7744047619047619, 'F1': 0.7647058823529411, 'F2': 0.7407467532467533, 'Recall': 0.7428571428571429, 'Precision': 0.7867965367965368, 'Average Precision': 0.7188940092165899, 'Specificity': 0.8055555555555556, 'NPV': 0.75, 'PR-AUC': 0.86662765078961}

Classifier: GaussianNB
Median performance: {'MCC': 0.49600925258068085, 'Balanced Accuracy': 0.7371031746031746, 'F1': 0.7048387096774194, 'F2': 0.6636952434013512, 'Recall': 0.6480158730158729, 'Precision': 0.7795138888888888, 'Average Precision': 0.6826984951181745, 'Specificity': 0.8285714285714286, 'NPV': 0.6956521739130435, 'PR-AUC': 0.8209972906113024}

Classifier: KNeighborsClassifier
Median performance: {'MCC': 0.4665694748158435, 'Balanced Accuracy': 0.7325396825396826, 'F1': 0.7338442419221209, 'F2': 0.7327287144414709, 'Recall': 0.7142857142857143, 'Precision': 0.7464285714285714, 'Average Pre

In [12]:
# Construct DFs per model out of the NestedCV output for the score metrics of each outer loop across all rounds executed
logreg_scores_u = dict_to_df(outer_loop_scores_u, 'LogisticRegression')
gnb_scores_u = dict_to_df(outer_loop_scores_u, 'GaussianNB')
knn_scores_u = dict_to_df(outer_loop_scores_u, 'KNeighborsClassifier')
lda_scores_u = dict_to_df(outer_loop_scores_u, 'LinearDiscriminantAnalysis')
svm_scores_u = dict_to_df(outer_loop_scores_u, 'SVM')

# Construct DFs per model out of the NestedCV output for the saved hyper-parameters across all rounds executed
logreg_params_u = dict_to_df(outer_params_u, 'LogisticRegression')
gnb_params_u = dict_to_df(outer_params_u, 'GaussianNB')
knn_params_u = dict_to_df(outer_params_u, 'KNeighborsClassifier')
lda_params_u = dict_to_df(outer_params_u, 'LinearDiscriminantAnalysis')
svm_params_u = dict_to_df(outer_params_u, 'SVM')

In [13]:
# Store all classifier data-frames in a list
all_dfs_u = [logreg_scores_u, gnb_scores_u, knn_scores_u, lda_scores_u, svm_scores_u]

# Store their corresponding names
classifier_names = ['Logistic Regression', 'Gaussian NB', 'k-NN', 'LDA', 'SVM']

In [14]:
scores_fig_u = metric_box_plots(all_dfs_u, classifier_names, 'Box Plots of Metric Scores Per Classifier (Raw dataset/Undersampling)', [-0.1, 1])
pio.show(scores_fig_u)

In [15]:
# Save as pdf
pio.write_image(scores_fig_u, "plots/scores-fig-under.pdf", height=1200, width=450)

##### Oversampling Method

In [16]:
# Creating an instance of SMOTE
smote = SMOTE()

# Balancing the data
X_o, y_o = smote.fit_resample(X, y)

Perform hyper-parameter tuning through n-CV

In [17]:
# Initialize the NestedCV class with the classifiers, parameter grids, rounds, seeds, and folds
nested_cv = NestedCV(classifiers=classifiers, param_grids=param_grids,
                     rounds=rounds, inner_cv_folds=3,
                     outer_cv_folds=5, seeds=seeds,
                     n_trials=50)

# Fit the nested cross-validation
outer_loop_scores_o, outer_params_o = nested_cv.fit(X_o, y_o)

# Print median performance for each classifier
print("Median performance per classifier:")
for clf_name in classifiers.keys():
    print(f"\nClassifier: {clf_name}")
    median_performance = {metric: np.median([result[metric] for result in outer_loop_scores_o[clf_name]])
                          for metric in outer_loop_scores_o[clf_name][0]}
    
    # Print median performance and best parameters
    print(f"Median performance: {median_performance}")


Using classifier: LogisticRegression



  0%|          | 0/10 [00:00<?, ?it/s]


Starting round 1...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 2...


...


Starting round 9...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 10...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t

Median performance per classifier:

Classifier: LogisticRegression
Median performance: {'MCC': 0.5378143267749169, 'Balanced Accuracy': 0.7679487179487179, 'F1': 0.758098223615465, 'F2': 0.7465025831543508, 'Recall': 0.7404428904428905, 'Precision': 0.7795138888888888, 'Average Precision': 0.7098547462253222, 'Specificity': 0.7878787878787878, 'NPV': 0.7519230769230769, 'PR-AUC': 0.8350376433247813}

Classifier: GaussianNB
Median performance: {'MCC': 0.5094681964468134, 'Balanced Accuracy': 0.7537878787878788, 'F1': 0.7450787401574803, 'F2': 0.7198044478527608, 'Recall': 0.7099067599067599, 'Precision': 0.7768199233716475, 'Average Precision': 0.6971080601779511, 'Specificity': 0.8015151515151515, 'NPV': 0.7379862700228832, 'PR-AUC': 0.8165749546885404}

Classifier: KNeighborsClassifier
Median performance: {'MCC': 0.5234673002944472, 'Balanced Accuracy': 0.7600815850815851, 'F1': 0.7719211822660099, 'F2': 0.8149679990339331, 'Recall': 0.8461538461538461, 'Precision': 0.7302494802494802

In [18]:
# Construct DFs per model out of the NestedCV output for the score metrics of each outer loop across all rounds executed
logreg_scores_o = dict_to_df(outer_loop_scores_o, 'LogisticRegression')
gnb_scores_o = dict_to_df(outer_loop_scores_o, 'GaussianNB')
knn_scores_o = dict_to_df(outer_loop_scores_o, 'KNeighborsClassifier')
lda_scores_o = dict_to_df(outer_loop_scores_o, 'LinearDiscriminantAnalysis')
svm_scores_o = dict_to_df(outer_loop_scores_o, 'SVM')

# Construct DFs per model out of the NestedCV output for the saved hyper-parameters across all rounds executed
logreg_params_o = dict_to_df(outer_params_o, 'LogisticRegression')
gnb_params_o = dict_to_df(outer_params_o, 'GaussianNB')
knn_params_o = dict_to_df(outer_params_o, 'KNeighborsClassifier')
lda_params_o = dict_to_df(outer_params_o, 'LinearDiscriminantAnalysis')
svm_params_o = dict_to_df(outer_params_o, 'SVM')

In [19]:
outer_loop_scores

{'LogisticRegression': [{'MCC': 0.3473915412083391,
   'Balanced Accuracy': 0.6603535353535354,
   'F1': 0.5396825396825397,
   'F2': 0.49707602339181284,
   'Recall': 0.4722222222222222,
   'Precision': 0.6296296296296297,
   'Average Precision': 0.48359961268458,
   'Specificity': 0.8484848484848485,
   'NPV': 0.7466666666666667,
   'PR-AUC': 0.6359627313886164},
  {'MCC': 0.5472275850985698,
   'Balanced Accuracy': 0.7411255411255411,
   'F1': 0.6551724137931034,
   'F2': 0.5828220858895705,
   'Recall': 0.5428571428571428,
   'Precision': 0.8260869565217391,
   'Average Precision': 0.6068630465531025,
   'Specificity': 0.9393939393939394,
   'NPV': 0.7948717948717948,
   'PR-AUC': 0.7820466899229309},
  {'MCC': 0.6221151817721777,
   'Balanced Accuracy': 0.8041125541125541,
   'F1': 0.7462686567164178,
   'F2': 0.7267441860465117,
   'Recall': 0.7142857142857143,
   'Precision': 0.78125,
   'Average Precision': 0.6570456152758133,
   'Specificity': 0.8939393939393939,
   'NPV': 0.8

In [20]:
# Store all classifier data-frames in a list
all_dfs_o = [logreg_scores_o, gnb_scores_o, knn_scores_o, lda_scores_o, svm_scores_o]

# Store their corresponding names
classifier_names = ['Logistic Regression', 'Gaussian NB', 'k-NN', 'LDA', 'SVM']

In [21]:
scores_fig_o = metric_box_plots(all_dfs_o, classifier_names, 'Box Plots of Metric Scores Per Classifier (Raw dataset/Oversampling)', [0, 1])
pio.show(scores_fig_o)

In [22]:
# Save as pdf
pio.write_image(scores_fig_o, "plots/scores-fig-over.pdf", height=1200, width=450)

##### *Perform nCV on the curated data*

In [23]:
# Load data
os.chdir('/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/')
data2 = pd.read_csv('data/Diabetes_curated.csv')
X2 = data2.iloc[:, 1:-1].values
y2 = data2.iloc[:, -1].values

In [24]:
# Initialize the NestedCV class with the classifiers, parameter grids, rounds, seeds, and folds
nested_cv = NestedCV(classifiers=classifiers, param_grids=param_grids,
                     rounds=rounds, inner_cv_folds=3,
                     outer_cv_folds=5, seeds=seeds,
                     n_trials=50)

# Fit the nested cross-validation
outer_loop_scores2, outer_params2 = nested_cv.fit(X2, y2)

# Print median performance for each classifier
print("Median performance per classifier:")
for clf_name in classifiers.keys():
    print(f"\nClassifier: {clf_name}")
    median_performance = {metric: np.median([result[metric] for result in outer_loop_scores2[clf_name]])
                          for metric in outer_loop_scores2[clf_name][0]}
    
    # Print median performance and best parameters
    print(f"Median performance: {median_performance}")



Using classifier: LogisticRegression



  0%|          | 0/10 [00:00<?, ?it/s]


Starting round 1...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 2...


...


Starting round 9...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



Starting round 10...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/gly

Median performance per classifier:

Classifier: LogisticRegression
Median performance: {'MCC': 0.4789455088967368, 'Balanced Accuracy': 0.7295673076923077, 'F1': 0.6153846153846154, 'F2': 0.5901741293532339, 'Recall': 0.554945054945055, 'Precision': 0.6666666666666666, 'Average Precision': 0.5096566831860949, 'Specificity': 0.875, 'NPV': 0.8285714285714286, 'PR-AUC': 0.735427974471077}

Classifier: GaussianNB
Median performance: {'MCC': 0.49259434678523517, 'Balanced Accuracy': 0.7505760368663594, 'F1': 0.6547619047619048, 'F2': 0.6251205865328961, 'Recall': 0.6153846153846154, 'Precision': 0.6547619047619048, 'Average Precision': 0.5291601255886971, 'Specificity': 0.8709677419354839, 'NPV': 0.8412298387096775, 'PR-AUC': 0.7377636065993124}

Classifier: KNeighborsClassifier
Median performance: {'MCC': 0.28183171497163995, 'Balanced Accuracy': 0.5961981566820276, 'F1': 0.3582887700534759, 'F2': 0.3077651515151515, 'Recall': 0.2857142857142857, 'Precision': 0.6, 'Average Precision': 0.39

In [25]:
# Construct DFs per model out of the NestedCV output for the score metrics of each outer loop across all rounds executed
logreg_scores2 = dict_to_df(outer_loop_scores2, 'LogisticRegression')
gnb_scores2 = dict_to_df(outer_loop_scores2, 'GaussianNB')
knn_scores2 = dict_to_df(outer_loop_scores2, 'KNeighborsClassifier')
lda_scores2 = dict_to_df(outer_loop_scores2, 'LinearDiscriminantAnalysis')
svm_scores2 = dict_to_df(outer_loop_scores2, 'SVM')

# Construct DFs per model out of the NestedCV output for the saved hyper-parameters across all rounds executed
logreg_params2 = dict_to_df(outer_params2, 'LogisticRegression')
gnb_params2 = dict_to_df(outer_params2, 'GaussianNB')
knn_params2 = dict_to_df(outer_params2, 'KNeighborsClassifier')
lda_params2 = dict_to_df(outer_params2, 'LinearDiscriminantAnalysis')
svm_params2 = dict_to_df(outer_params2, 'SVM')

In [26]:
# Store all classifier data-frames in a list
all_dfs2 = [logreg_scores2, gnb_scores2, knn_scores2, lda_scores2, svm_scores2]

# Store their corresponding names
classifier_names = ['Logistic Regression', 'Gaussian NB', 'k-NN', 'LDA', 'SVM']


In [27]:
scores_fig2 = metric_box_plots(all_dfs2, classifier_names, 'Box Plots of Metric Scores Per Classifier (Curated)', [-0.2, 1])
pio.show(scores_fig2)

In [28]:
# Save as pdf
pio.write_image(scores_fig2, "plots/scores-fig2.pdf", height=1200, width=450)

Try using the methods of under and oversampling for hyperparameter tuning with optuna through nested cross-validation.

#### *Perform nCV on the curated data with balanced classes*

##### Undersampling Method

In [29]:
# Create a RandomUnderSampler object
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# Balancing the data (using the curated dataset)
X_u2, y_u2 = rus.fit_resample(X2, y2)

Perform hyper-parameter tuning through n-CV

In [30]:
# Initialize the NestedCV class with the classifiers, parameter grids, rounds, seeds, and folds
nested_cv = NestedCV(classifiers=classifiers, param_grids=param_grids,
                     rounds=rounds, inner_cv_folds=3,
                     outer_cv_folds=5, seeds=seeds,
                     n_trials=50)

# Fit the nested cross-validation
outer_loop_scores_u2, outer_params_u2 = nested_cv.fit(X_u, y_u)

# Print median performance for each classifier
print("Median performance per classifier:")
for clf_name in classifiers.keys():
    print(f"\nClassifier: {clf_name}")
    median_performance = {metric: np.median([result[metric] for result in outer_loop_scores_u2[clf_name]])
                          for metric in outer_loop_scores_u2[clf_name][0]}
    
    # Print median performance and best parameters
    print(f"Median performance: {median_performance}")


Using classifier: LogisticRegression



  0%|          | 0/10 [00:00<?, ?it/s]


Starting round 1...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 2...


...


Starting round 9...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 10...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t

Median performance per classifier:

Classifier: LogisticRegression
Median performance: {'MCC': 0.5512273465625287, 'Balanced Accuracy': 0.7744047619047619, 'F1': 0.7647058823529411, 'F2': 0.7506420545746388, 'Recall': 0.7428571428571429, 'Precision': 0.7867965367965368, 'Average Precision': 0.7188940092165899, 'Specificity': 0.8055555555555556, 'NPV': 0.7533783783783784, 'PR-AUC': 0.86662765078961}

Classifier: GaussianNB
Median performance: {'MCC': 0.48415440692420075, 'Balanced Accuracy': 0.7313492063492064, 'F1': 0.7086848635235732, 'F2': 0.6647398843930636, 'Recall': 0.6480158730158729, 'Precision': 0.7777777777777778, 'Average Precision': 0.6807812241413851, 'Specificity': 0.8285714285714286, 'NPV': 0.6965894465894467, 'PR-AUC': 0.8209972906113024}

Classifier: KNeighborsClassifier
Median performance: {'MCC': 0.4765393553965823, 'Balanced Accuracy': 0.7376984126984127, 'F1': 0.7352941176470589, 'F2': 0.7265005846433676, 'Recall': 0.7142857142857143, 'Precision': 0.75, 'Average Pre

In [31]:
# Construct DFs per model out of the NestedCV output for the score metrics of each outer loop across all rounds executed
logreg_scores_u2 = dict_to_df(outer_loop_scores_u2, 'LogisticRegression')
gnb_scores_u2 = dict_to_df(outer_loop_scores_u2, 'GaussianNB')
knn_scores_u2 = dict_to_df(outer_loop_scores_u2, 'KNeighborsClassifier')
lda_scores_u2 = dict_to_df(outer_loop_scores_u2, 'LinearDiscriminantAnalysis')
svm_scores_u2 = dict_to_df(outer_loop_scores_u2, 'SVM')

# Construct DFs per model out of the NestedCV output for the saved hyper-parameters across all rounds executed
logreg_params_u2 = dict_to_df(outer_params_u2, 'LogisticRegression')
gnb_params_u2 = dict_to_df(outer_params_u2, 'GaussianNB')
knn_params_u2 = dict_to_df(outer_params_u2, 'KNeighborsClassifier')
lda_params_u2 = dict_to_df(outer_params_u2, 'LinearDiscriminantAnalysis')
svm_params_u2 = dict_to_df(outer_params_u2, 'SVM')

In [32]:
# Store all classifier data-frames in a list
all_dfs_u2 = [logreg_scores_u2, gnb_scores_u2, knn_scores_u2, lda_scores_u2, svm_scores_u2]

# Store their corresponding names
classifier_names = ['Logistic Regression', 'Gaussian NB', 'k-NN', 'LDA', 'SVM']

In [33]:
scores_fig_u2 = metric_box_plots(all_dfs_u2, classifier_names, 'Box Plots of Metric Scores Per Classifier (Curated/Undersampling)', [-0.1, 1])
pio.show(scores_fig_u2)

In [34]:
# Save as pdf
pio.write_image(scores_fig_u2, "plots/scores-fig-under2.pdf", height=1200, width=450)

##### Oversampling Method

In [35]:
# Creating an instance of SMOTE
smote = SMOTE()

# Balancing the data
X_o2, y_o2 = smote.fit_resample(X2, y2)

Perform hyper-parameter tuning through n-CV

In [36]:
# Initialize the NestedCV class with the classifiers, parameter grids, rounds, seeds, and folds
nested_cv = NestedCV(classifiers=classifiers, param_grids=param_grids,
                     rounds=rounds, inner_cv_folds=3,
                     outer_cv_folds=5, seeds=seeds,
                     n_trials=50)

# Fit the nested cross-validation
outer_loop_scores_o2, outer_params_o2 = nested_cv.fit(X_o, y_o)

# Print median performance for each classifier
print("Median performance per classifier:")
for clf_name in classifiers.keys():
    print(f"\nClassifier: {clf_name}")
    median_performance = {metric: np.median([result[metric] for result in outer_loop_scores_o2[clf_name]])
                          for metric in outer_loop_scores_o[clf_name][0]}
    
    # Print median performance and best parameters
    print(f"Median performance: {median_performance}")


Using classifier: LogisticRegression



  0%|          | 0/10 [00:00<?, ?it/s]


Starting round 1...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 2...


...


Starting round 9...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t


Starting round 10...


/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

/Users/glykeriasp/Documents/DSIT/Machine Learning in Computational Biology/Assignments/Assignment 2/Assignment_2/src/nestedCV.py:60: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in t

Median performance per classifier:

Classifier: LogisticRegression
Median performance: {'MCC': 0.5340924991575906, 'Balanced Accuracy': 0.7651515151515151, 'F1': 0.758098223615465, 'F2': 0.7465025831543508, 'Recall': 0.7404428904428905, 'Precision': 0.7795138888888888, 'Average Precision': 0.7098547462253222, 'Specificity': 0.7878787878787878, 'NPV': 0.7519230769230769, 'PR-AUC': 0.8348884238573799}

Classifier: GaussianNB
Median performance: {'MCC': 0.5135766801619671, 'Balanced Accuracy': 0.755944055944056, 'F1': 0.7410464820929642, 'F2': 0.7197566649781431, 'Recall': 0.7099067599067599, 'Precision': 0.7736875800256082, 'Average Precision': 0.7014046233941359, 'Specificity': 0.7878787878787878, 'NPV': 0.7362318840579709, 'PR-AUC': 0.8158044384330072}

Classifier: KNeighborsClassifier
Median performance: {'MCC': 0.533523140109676, 'Balanced Accuracy': 0.7644522144522145, 'F1': 0.7799647621444752, 'F2': 0.8137814779138572, 'Recall': 0.8461538461538461, 'Precision': 0.7330316742081449, 

In [37]:
# Construct DFs per model out of the NestedCV output for the score metrics of each outer loop across all rounds executed
logreg_scores_o2 = dict_to_df(outer_loop_scores_o2, 'LogisticRegression')
gnb_scores_o2 = dict_to_df(outer_loop_scores_o2, 'GaussianNB')
knn_scores_o2 = dict_to_df(outer_loop_scores_o2, 'KNeighborsClassifier')
lda_scores_o2 = dict_to_df(outer_loop_scores_o2, 'LinearDiscriminantAnalysis')
svm_scores_o2 = dict_to_df(outer_loop_scores_o2, 'SVM')

# Construct DFs per model out of the NestedCV output for the saved hyper-parameters across all rounds executed
logreg_params_o2 = dict_to_df(outer_params_o2, 'LogisticRegression')
gnb_params_o2 = dict_to_df(outer_params_o2, 'GaussianNB')
knn_params_o2 = dict_to_df(outer_params_o2, 'KNeighborsClassifier')
lda_params_o2 = dict_to_df(outer_params_o2, 'LinearDiscriminantAnalysis')
svm_params_o2 = dict_to_df(outer_params_o2, 'SVM')

In [38]:
# Store all classifier data-frames in a list
all_dfs_o2 = [logreg_scores_o2, gnb_scores_o2, knn_scores_o2, lda_scores_o2, svm_scores_o2]

# Store their corresponding names
classifier_names = ['Logistic Regression', 'Gaussian NB', 'k-NN', 'LDA', 'SVM']

In [39]:
scores_fig_o2 = metric_box_plots(all_dfs_o2, classifier_names, 'Box Plots of Metric Scores Per Classifier (Curated/Oversampling)', [0, 1])
pio.show(scores_fig_o2)

In [40]:
# Save as pdf
pio.write_image(scores_fig_o2, "plots/scores-fig-over2.pdf", height=1200, width=450)

### Selection of the best model

At this point we want to prioritize a set of three different metrics in order to decide upon the classifier and its parameters.



PR-AUC: Represents the area under the Precision-Recall curve, capturing the trade-off between precision and recall across different thresholds. This is particularly useful in imbalanced datasets, as it gives a comprehensive view of how the model performs with varying decision thresholds.

##### Models tuned using the raw dataset

In [41]:
# Priorities for the imbalanced data
priorities_ib = ['Recall', 'Precision', 'PR-AUC']

# Priorities for balanced data
priorities_b = ['Balanced Accuracy', 'Recall', 'Precision']

In [42]:
best_classifier, ci_dict, ci_widths, median_metrics = evaluate_clfs(all_dfs, classifier_names, priorities_ib, 2)

Overall Scores of Classifiers based on Priority Metrics:
Logistic Regression: 2.0108244808714337
Gaussian NB: 1.947797335598792
k-NN: 1.87098344027125
LDA: 2.0242885017634413
SVM: 1.2501739479599656


In [43]:
best_classifier

'LDA'

In [44]:
ci_dict

{'Logistic Regression': {'MCC': (0.4811086224599054, 0.5303783253136064),
  'Balanced Accuracy': (0.7246773259259572, 0.748844374595743),
  'F1': (0.6340025316141153, 0.6694077232547246),
  'F2': (0.590533408306007, 0.6291106799664026),
  'Recall': (0.5647950800426718, 0.6059033326557408),
  'Precision': (0.720488769829305, 0.7616905650693839),
  'Average Precision': (0.5635173890659819, 0.598266879143085),
  'Specificity': (0.8777776455046231, 0.8985673428403649),
  'NPV': (0.7914316685873479, 0.8084112498640756),
  'PR-AUC': (0.7069082575474879, 0.7456852183195928)},
 'LDA': {'MCC': (0.4776733377301878, 0.5293120791293736),
  'Balanced Accuracy': (0.7222767004831853, 0.747212144005659),
  'F1': (0.6301497997428234, 0.6670024326743518),
  'F2': (0.5850517084473361, 0.6243568837720033),
  'Recall': (0.5584431329078061, 0.5999378194731462),
  'Precision': (0.7210761770884335, 0.7647357553506429),
  'Average Precision': (0.5617167732717724, 0.5981359163140195),
  'Specificity': (0.879485

##### Models tuned using the raw dataset / Undersampling

In [45]:
best_classifier_u, ci_dict_u, ci_widths_u, median_metrics_u = evaluate_clfs(all_dfs_u, classifier_names, priorities_b, 2)

Overall Scores of Classifiers based on Priority Metrics:
Logistic Regression: 2.3040584415584417
Gaussian NB: 2.164632936507936
k-NN: 2.1932539682539685
LDA: 2.2860379000084885
SVM: 2.109171920800841


In [46]:
best_classifier_u

'Logistic Regression'

In [47]:
ci_dict_u

{'Logistic Regression': {'MCC': (0.5096810607473777, 0.5605847214194554),
  'Balanced Accuracy': (0.7532404926052408, 0.778632523267775),
  'F1': (0.7464227083643044, 0.7732642534854278),
  'F2': (0.7332291558913159, 0.765724960322571),
  'Recall': (0.7243832476457966, 0.7626961174335686),
  'Precision': (0.7654420808061076, 0.7976273085400573),
  'Average Precision': (0.6961807298945779, 0.7234778895361706),
  'Specificity': (0.7685823941377695, 0.8080842725288974),
  'NPV': (0.7424920026973906, 0.7713207874735754),
  'PR-AUC': (0.8486121487141103, 0.8726770287638861)},
 'LDA': {'MCC': (0.5061994555687859, 0.5573553765755994),
  'Balanced Accuracy': (0.7514360629406052, 0.7770401275355854),
  'F1': (0.7414663400185247, 0.7696995562244607),
  'F2': (0.7241029156533694, 0.7585989885688347),
  'Recall': (0.7128803657243369, 0.7528339199899489),
  'Precision': (0.7691174649179272, 0.7989330560411944),
  'Average Precision': (0.6957536136852056, 0.7223658086488778),
  'Specificity': (0.778

##### Models tuned using the raw dataset / Oversampling

In [48]:
best_classifier_o, ci_dict_o, ci_widths_o, median_metrics_o = evaluate_clfs(all_dfs_o, classifier_names, priorities_b, 2)

Overall Scores of Classifiers based on Priority Metrics:
Logistic Regression: 2.287905497280497
Gaussian NB: 2.240514562066286
k-NN: 2.3364849114849116
LDA: 2.27101128038628
SVM: 2.294911326490274


In [49]:
best_classifier_o

'k-NN'

In [50]:
ci_dict_o

{'k-NN': {'MCC': (0.513123976925574, 0.5573748456328536),
  'Balanced Accuracy': (0.7526522312030182, 0.7752871627363758),
  'F1': (0.7702379374297084, 0.7898137645430421),
  'F2': (0.8015850735931551, 0.8226415121249757),
  'Recall': (0.8228318343107518, 0.8483756248967075),
  'Precision': (0.7202647812841064, 0.7466236786564228),
  'Average Precision': (0.6835208255754334, 0.7070582474435457),
  'Specificity': (0.6720189753340315, 0.7126523533372973),
  'NPV': (0.7968763555492887, 0.8217741676792375),
  'PR-AUC': (0.8425414411854296, 0.8637187247806973)},
 'SVM': {'MCC': (0.5421677421026649, 0.586044454836417),
  'Balanced Accuracy': (0.75586347572512, 0.7814185755569311),
  'F1': (0.7063148105033714, 0.749618175841753),
  'F2': (0.6405027758111346, 0.6987168576733098),
  'Recall': (0.6046804969272155, 0.6704523702056517),
  'Precision': (0.8558468770768441, 0.8947028837897291),
  'Average Precision': (0.7223926663725402, 0.7481692440825858),
  'Specificity': (0.8809161150629543, 0.9

##### Models tuned using the curated dataset

In [51]:
best_classifier2, ci_dict2, ci_widths2, median_metrics2 = evaluate_clfs(all_dfs2, classifier_names, priorities_ib, 5)

Overall Scores of Classifiers based on Priority Metrics:
Logistic Regression: 1.9570396960827987
Gaussian NB: 2.0079101267458324
k-NN: 1.46131050104826
LDA: 2.049180767751281
SVM: 0.3996401280903543


In [52]:
best_classifier2

'LDA'

In [53]:
ci_dict2

{'Logistic Regression': {'MCC': (0.4191991469348981, 0.500527487831441),
  'Balanced Accuracy': (0.6969816856420786, 0.7377701754001053),
  'F1': (0.5655601445798579, 0.631715035713263),
  'F2': (0.5360590649996144, 0.6089786176452161),
  'Recall': (0.5192717817130885, 0.5976512952099884),
  'Precision': (0.6294984068171474, 0.6983196019714497),
  'Average Precision': (0.4832962522702118, 0.536892861155327),
  'Specificity': (0.8605913186592632, 0.891989326502027),
  'NPV': (0.8130782392057592, 0.8378782755417951),
  'PR-AUC': (0.6894912012061475, 0.746628752357963)},
 'Gaussian NB': {'MCC': (0.45228685010254044, 0.5412844741124729),
  'Balanced Accuracy': (0.7202040755880468, 0.7630385688642753),
  'F1': (0.6029535539774905, 0.669920093742779),
  'F2': (0.5887567061610788, 0.658298714137252),
  'Recall': (0.5803234915573288, 0.6537424425086054),
  'Precision': (0.6308014782113353, 0.7085616129458733),
  'Average Precision': (0.5093580570767494, 0.5653040040898368),
  'Specificity': (0

##### Models tuned using the curated dataset / Undersampling

In [54]:
best_classifier_u2, ci_dict_u2, ci_widths_u2, median_metrics_u2 = evaluate_clfs(all_dfs_u2, classifier_names, priorities_b, 5)

Overall Scores of Classifiers based on Priority Metrics:
Logistic Regression: 2.3040584415584417
Gaussian NB: 2.157142857142857
k-NN: 2.201984126984127
LDA: 2.2860379000084885
SVM: 2.1177866107683387


In [55]:
best_classifier_u2

'Logistic Regression'

In [56]:
ci_dict_u2

{'Logistic Regression': {'MCC': (0.5129026911356446, 0.5629717164458007),
  'Balanced Accuracy': (0.7548497171007753, 0.7798486955976371),
  'F1': (0.7480237484410118, 0.7746057999843047),
  'F2': (0.73481971558017, 0.7673752759350921),
  'Recall': (0.726003756805792, 0.7645041797021441),
  'Precision': (0.7670062739020463, 0.7984703119441939),
  'Average Precision': (0.6977842929202862, 0.7245587595267418),
  'Specificity': (0.7700705938689241, 0.8088182950199652),
  'NPV': (0.7440470680683088, 0.7729246839675016),
  'PR-AUC': (0.8486821217383307, 0.8728280197400948)},
 'Gaussian NB': {'MCC': (0.44627390023739166, 0.49877717167985214),
  'Balanced Accuracy': (0.7189587890253522, 0.7450253379587748),
  'F1': (0.6921949750322519, 0.7229152204530166),
  'F2': (0.6546752006946702, 0.6913549061420828),
  'Recall': (0.6319146269768525, 0.6732917222294965),
  'Precision': (0.7615738547739556, 0.7971184696874404),
  'Average Precision': (0.6692038447534904, 0.696872436411461),
  'Specificity'

##### Models tuned using the curated dataset / Oversampling

In [57]:
best_classifier_o2, ci_dict_o2, ci_widths_o2, median_metrics_o2 = evaluate_clfs(all_dfs_o2, classifier_names, priorities_b, 5)

Overall Scores of Classifiers based on Priority Metrics:
Logistic Regression: 2.2851082944832943
Gaussian NB: 2.239538395876424
k-NN: 2.3436377348142052
LDA: 2.27101128038628
SVM: 2.281863506863507


In [58]:
best_classifier_o2

'k-NN'

In [59]:
ci_dict_o2

{'Logistic Regression': {'MCC': (0.5001247642553903, 0.5432208699906564),
  'Balanced Accuracy': (0.7491183208853404, 0.7706299308629111),
  'F1': (0.7428840506146855, 0.7657979077927103),
  'F2': (0.7311832770813831, 0.7587082765680639),
  'Recall': (0.7233919139195627, 0.7550975965699481),
  'Precision': (0.7602397333813327, 0.7849290416878452),
  'Average Precision': (0.6914283817800126, 0.712941144919353),
  'Specificity': (0.7658402442537353, 0.7951667487532579),
  'NPV': (0.7388679628123102, 0.7632017241705383),
  'PR-AUC': (0.8138085223300496, 0.8431101618998048)},
 'Gaussian NB': {'MCC': (0.4783592478346978, 0.5254039149523682),
  'Balanced Accuracy': (0.7377341296264398, 0.7610630731707627),
  'F1': (0.7248243271558774, 0.7498704513228378),
  'F2': (0.703501502297281, 0.7308192728480025),
  'Recall': (0.6897334238280206, 0.7194880213934249),
  'Precision': (0.7611940361438814, 0.7894000142122954),
  'Average Precision': (0.6830474394899624, 0.7073173512005284),
  'Specificity'

Creating barplot graphs with the median metric scores per classifiers for the six different experiments.

In [60]:
meds1 = plot_median_scores(median_metrics, 'Median Scores Across all Classifiers (Raw dataset)')
meds1.show()
pio.write_image(meds1, "plots/medians1.pdf", height=400, width=750)

In [61]:
meds_u = plot_median_scores(median_metrics_u, 'Median Scores Across all Classifiers (Raw dataset/Undersampling)')
meds_u.show()
pio.write_image(meds_u, "plots/medians-u.pdf", height=400, width=750)

In [62]:
meds_o = plot_median_scores(median_metrics_o, 'Median Scores Across all Classifiers (Raw dataset/Oversampling)')
meds_o.show()
pio.write_image(meds_o, "plots/medians-o.pdf", height=400, width=750)

In [63]:
meds2 = plot_median_scores(median_metrics2, 'Median Scores Across all Classifiers (Curated dataset)')
meds2.show()
pio.write_image(meds2, "plots/medians2.pdf", height=400, width=750)

In [64]:
meds_u2 = plot_median_scores(median_metrics_u2, 'Median Scores Across all Classifiers (Curated dataset/Undersampling)')
meds_u2.show()
pio.write_image(meds_u2, "plots/medians-u2.pdf", height=400, width=750)

In [65]:
meds_o2 = plot_median_scores(median_metrics_o2, 'Median Scores Across all Classifiers (Curated dataset/Oversampling)')
meds_o2.show()
pio.write_image(meds_o2, "plots/medians-o2.pdf", height=400, width=750)

As seen above, class balancing using oversampling and undersampling in the curated dataset yielded the same results as the imbalanced curated dataset.

### Final model selection

As shown above, the experiments in which the models managed to showcase an overall better performance across the majority of the metrics were the once were data preprocessing was applied.
More specifically, better results were achieved upon using the oversampling method for class balancing.

For the evaluation of each classifier, the sum of the priority metrics was used to compare their performance.
As seen above, across the different experiments, there was not a consensus for the best algorithm.
However, LDA appear one time (2), k-NN two times (2), Logistic Regression two times (2).
SVM was one of the algorithms that had the worst performances and is the first to be eliminated.

Logistic Regression and LDA appear to have very similar metric scores throughout the different experiments.
Finally, Logistic Regression is selected as a final model to be trained, since it shows a very good overall performace across all different experiments, and is suspected to yield better results on unseen data.
Furthermore Logistic Regression managed to receive first place in both curated and uncurated experiments, explaining probability less sensitivity to outliers in a dataset, making it more robust.

Overall, Logistic Regression doesn't assume a linear relationship between the features of the dataset, while it can also avoid overfitting through the L1 and L2 regularization terms.